In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'/Users/wiamboumaazi/Downloads/clean_tweet_dataset.csv')
df.dropna(inplace = True)
df[['no_followers','no_followings', 'no_userfavorites','no_lists' , 'no_tweets','no_retweets','no_favorites', 'no_hashtags', 'no_usermentions','no_urls', 'label']]=df[['no_followers','no_followings', 'no_userfavorites','no_lists' , 'no_tweets','no_retweets','no_favorites', 'no_hashtags', 'no_usermentions','no_urls', 'label']].astype(int)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,tweet,tweet_clean,no_followers,no_followings,no_userfavorites,no_lists,no_tweets,no_retweets,no_favorites,no_hashtags,no_usermentions,no_urls,label
0,That just seriously ruined my night..,seriously ruined night,163,139,1560,1,25985,0,0,0,0,0,0
1,@JessleaC I no they are so funny ..! I love yo...,jessleac funny love pictures take really good ...,113,197,1072,1,2142,0,0,0,1,0,0
2,@honkwas might drive so we could go subway or ...,honkwas might drive could go subway mcdonalds ...,168,382,1732,1,3493,0,0,0,1,0,0
3,That @brad_frost post is bourne out by my expe...,bradfrost post bourne experience current proje...,285,596,5765,45,8670,0,0,0,1,0,0
4,why tf do i keep waking up every 2 hours -.-,tf keep waking every hours,631,353,1589,2,12285,0,0,0,0,0,0


In [4]:
data = df.drop(['tweet', 'tweet_clean'] , axis =1)
data.head()

,no_followers,no_followings,no_userfavorites,no_lists,no_tweets,no_retweets,no_favorites,no_hashtags,no_usermentions,no_urls,label
0,163,139,1560,1,25985,0,0,0,0,0,0
1,113,197,1072,1,2142,0,0,0,1,0,0
2,168,382,1732,1,3493,0,0,0,1,0,0
3,285,596,5765,45,8670,0,0,0,1,0,0
4,631,353,1589,2,12285,0,0,0,0,0,0


In [5]:
from sklearn.model_selection import train_test_split
#X_data is the data without the label column
X_data = data.drop('label', axis=1)
#Y_data is the label we want to train our data for 
y_data = data['label']
# Random state is used then you can guarantee that the output of Run 1 will be equal to the output of Run 2
#test size is 30% of the training data
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, shuffle=True, random_state=27)

In [6]:
from sklearn.ensemble import RandomForestClassifier
# creating the RF classifier model using the RandomForestClassifier from scikit learn
#n_estimators= num of trees, max_features=mtry, min_sample_split= Min no. of samples required to
#split
# Minimum number of samples required at each leaf node
model = RandomForestClassifier(n_estimators=500, max_features='auto', max_depth=70, bootstrap=False)



In [8]:
model.fit(X_train, y_train).predict_proba(X_test)

array([[0.904, 0.096],
       [0.   , 1.   ],
       [0.922, 0.078],
       ...,
       [0.   , 1.   ],
       [0.938, 0.062],
       [0.984, 0.016]])

In [11]:
from numpy import sqrt
from numpy import argmax
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
#with the best threshold

#predict yields the estimated probability of event according to the fitted model
y_preds = model.predict_proba(X_test)

preds = y_preds[:,1]
#generate the ROC curve with different thresholds [0 to 1] 
fpr, tpr, thresholds = metrics.roc_curve(y_test, preds)

# auc_score = metrics.auc(fpr, tpr)
#roc_auc_score will compute the classification based on the given threshold
#if preds<th then negative otherwise positive
auc_score = roc_auc_score(y_test, preds)

# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
index = argmax(gmeans)

print("threshold: ", thresholds[index])
print("auc: ", auc_score)
print("g_mean: ", gmeans[index])
print("sensitivity: ", tpr[index])
print("specificity: ", 1-fpr[index])

threshold:  0.2030909090909091
auc:  0.8953048527774468
g_mean:  0.8269126363902423
sensitivity:  0.776705288229079
specificity:  0.880365459827007


In [7]:
from numpy import sqrt
from numpy import argmax
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold

#5-fold stratified cross-validation to ensure that each fold is representative of the actual dataset. 
#The train-test ratio is 70%:30% of the entire dataset.
#Cross validation is done on the same 70% data of the training set.
kf = StratifiedKFold(n_splits=3, random_state=27, shuffle=True)
#create list to hold the value of the following for each block of testing data
cross_val_accuracy_lst = []
cross_val_g_mean_lst = [] 
cross_val_auc_lst = []
cross_val_sensitivity_lst = [] 
cross_val_specificity_lst = [] 

for train_index_ls, validation_index_ls in kf.split(X_train, y_train):
    # keeping validation set apart
    #train: refer to each sample in each block of training data selected within K-5 CV
    #validation: refer to samples designed for testing in the reamining block for each of the K-5 CV
    
    train, validation = X_train.iloc[train_index_ls], X_train.iloc[validation_index_ls]
    #target_train:refer to label we want to predict in training block
    #target value: refer to label in testing block
    target_train, target_val = y_train.iloc[train_index_ls], y_train.iloc[validation_index_ls]
    
    # training the model on oversampled 4 folds of training set
    model.fit(train, target_train).predict_proba(validation)
    # testing on 1 fold of validation set
    validation_preds = model.predict_proba(validation)
    
    preds = validation_preds[:,1]
    fpr, tpr, thresholds = metrics.roc_curve(target_val, preds)

    auc_score = roc_auc_score(target_val, preds)

    # calculate the g-mean for each threshold
    gmeans = sqrt(tpr * (1-fpr))
    # locate the index of the largest g-mean
    index = argmax(gmeans)
    
#     print(thresholds[ix])
    cross_val_g_mean_lst.append(gmeans[index])
    cross_val_auc_lst.append(auc_score)
    cross_val_sensitivity_lst.append(tpr[index])
    cross_val_specificity_lst.append(1-fpr[index])

print ('Cross validated g-mean: {}'.format(np.mean(cross_val_g_mean_lst)))
print ('Cross validated auc: {}'.format(np.mean(cross_val_auc_lst)))
print ('Cross validated sensitivity: {}'.format(np.mean(cross_val_sensitivity_lst)))
print ('Cross validated specificity: {}'.format(np.mean(cross_val_specificity_lst)))


print ('-- Cross validated std auc: {}'.format(np.std(cross_val_auc_lst)))
print ('-- Cross validated std g-mean: {}'.format(np.std(cross_val_g_mean_lst)))
print ('-- Cross validated std sensitivity: {}'.format(np.std(cross_val_sensitivity_lst)))
print ('-- Cross validated std specificity: {}'.format(np.std(cross_val_specificity_lst)))

Cross validated g-mean: 0.8193273077850968
Cross validated auc: 0.8911138894429147
Cross validated sensitivity: 0.7713415104528273
Cross validated specificity: 0.8703581617757212
-- Cross validated std auc: 0.000626479842541008
-- Cross validated std g-mean: 0.001514888874756878
-- Cross validated std sensitivity: 0.006236293964879074
-- Cross validated std specificity: 0.007735528740329789


In [10]:
#creating this data frame to show the importances of different features when fitting the model
#X_data: all column excluding the label
#importance from high to low
model.fit(X_train, y_train).predict_proba(X_test) 
pd.DataFrame({'Variable':X_data.columns,
              'Importance':model.feature_importances_}).sort_values('Importance', ascending=False)

,Variable,Importance
0,no_followers,0.201564
7,no_hashtags,0.176390
4,no_tweets,0.144020
2,no_userfavorites,0.134861
1,no_followings,0.133611
3,no_lists,0.079166
8,no_usermentions,0.046698
5,no_retweets,0.042584
9,no_urls,0.035185
6,no_favorites,0.005922
